In [ ]:
"""import pandas as pd
from sklearn import preprocessing
import numpy as np"""

In [ ]:
#data_points = 20

In [2]:
"""def get_data(csv):
    data = pd.read_csv(csv)
    data = data.drop('date', axis=1)
    data = data.drop(0, axis=0)
    data = data.values
    
    data_normaliser = preprocessing.MinMaxScaler()
    data_normalised = data_normaliser.fit_transform(data)
    
    #predict future patterns using past data_points
    points_normalised = np.array([data_normalised[i:i + data_points].copy() for i in range(len(data_normalised) - data_points)])
    open_values_normalised = np.array([data_normalised[:, 0][i + data_points].copy() for i in range(len(data_normalised) - data_points)])
    open_values_normalised = np.expand_dims(open_values_normalised, -1)

    open_values = np.array([data[:, 0][i + data_points].copy() for i in range(len(data) - data_points)])
    open_values = np.expand_dims(open_values, -1)

    normaliser = preprocessing.MinMaxScaler()
    normaliser.fit(open_values)

    def calc_ema(values, timeline):
        sma = np.mean(values[:, 3])
        ema = [sma]
        k = 2 / (1 + timeline)
        for i in range(len(past_price) - timeline, len(his)):
            close = past_price[i][3]
            ema.append(close * k + ema[-1] * (1 - k))
        return ema[-1]

    indicators = []
    for past_price in points_normalised:
        sma = np.mean(past_price[:, 3])
        macd = calculate_ema(past_price, 12) - calculate_ema(past_price, 26)
        indicators.append(np.array([sma]))

    indicators = np.array(indicators)

    scaler = preprocessing.MinMaxScaler()
    indicators_normalised = scaler.fit_transform(indicators)

    assert points_normalised.shape[0] == open_values_normalised.shape[0] == indicators_normalised.shape[0]
    return points_normalised, indicators_normalised, open_values_normalised, open_values, normaliser


def get_large_data(test_set_name):
    import os
    past_points = 0
    indicators = 0
    open_values = 0
    for csv_file_path in list(filter(lambda x: x.endswith('daily.csv'), os.listdir('./Users\Josh\Documents\4th Year Docs\Dist+Con\CA Group\StockcsvsStockcsvs'))):
        if not csv_file_path == test_set_name:
            print(csv_file_path)
            if type(past_points) == int:
                past_points, indicators, open_values, _, _ = get_data(csv_file_path)
            else:
                a, b, c, _, _ = get_data(csv_file_path)
                past_points = np.concatenate((past_points, a), 0)
                indicators = np.concatenate((indicators, b), 0)
                open_values = np.concatenate((open_values, c), 0)

    train_past_points = past_points
    train_indicator = indicators
    training = open_values

    past_points_test, indicator_test, test, unscaled_test, normaliser = get_data(test_set_name)

    return train_past_points, train_indicator, training, past_points_test, indicator_test, test, unscaled_test, normaliser"""

"def get_data(csv):\n    data = pd.read_csv(csv)\n    data = data.drop('date', axis=1)\n    data = data.drop(0, axis=0)\n    data = data.values\n    \n    data_normaliser = preprocessing.MinMaxScaler()\n    data_normalised = data_normaliser.fit_transform(data)\n    \n    #predict future patterns using past data_points\n    points_normalised = np.array([data_normalised[i:i + data_points].copy() for i in range(len(data_normalised) - data_points)])\n    open_values_normalised = np.array([data_normalised[:, 0][i + data_points].copy() for i in range(len(data_normalised) - data_points)])\n    open_values_normalised = np.expand_dims(open_values_normalised, -1)\n\n    open_values = np.array([data[:, 0][i + data_points].copy() for i in range(len(data) - data_points)])\n    open_values = np.expand_dims(open_values, -1)\n\n    normaliser = preprocessing.MinMaxScaler()\n    normaliser.fit(open_values)\n\n    def calc_ema(values, timeline):\n        sma = np.mean(values[:, 3])\n        ema = [sma]\

In [3]:
import pandas as pd
from sklearn import preprocessing
import numpy as np

history_points = 50


def csv_to_dataset(csv_path):
    data = pd.read_csv(csv_path)
    data = data.drop('date', axis=1)
    data = data.drop(0, axis=0)

    data = data.values

    data_normaliser = preprocessing.MinMaxScaler()
    data_normalised = data_normaliser.fit_transform(data)

    # using the last {history_points} open close high low volume data points, predict the next open value
    ohlcv_histories_normalised = np.array([data_normalised[i:i + history_points].copy() for i in range(len(data_normalised) - history_points)])
    next_day_open_values_normalised = np.array([data_normalised[:, 0][i + history_points].copy() for i in range(len(data_normalised) - history_points)])
    next_day_open_values_normalised = np.expand_dims(next_day_open_values_normalised, -1)

    next_day_open_values = np.array([data[:, 0][i + history_points].copy() for i in range(len(data) - history_points)])
    next_day_open_values = np.expand_dims(next_day_open_values, -1)

    y_normaliser = preprocessing.MinMaxScaler()
    y_normaliser.fit(next_day_open_values)

    def calc_ema(values, time_period):
        # https://www.investopedia.com/ask/answers/122314/what-exponential-moving-average-ema-formula-and-how-ema-calculated.asp
        sma = np.mean(values[:, 3])
        ema_values = [sma]
        k = 2 / (1 + time_period)
        for i in range(len(his) - time_period, len(his)):
            close = his[i][3]
            ema_values.append(close * k + ema_values[-1] * (1 - k))
        return ema_values[-1]

    technical_indicators = []
    for his in ohlcv_histories_normalised:
        # note since we are using his[3] we are taking the SMA of the closing price
        sma = np.mean(his[:, 3])
        macd = calc_ema(his, 12) - calc_ema(his, 26)
        technical_indicators.append(np.array([sma]))
        # technical_indicators.append(np.array([sma,macd,]))

    technical_indicators = np.array(technical_indicators)

    tech_ind_scaler = preprocessing.MinMaxScaler()
    technical_indicators_normalised = tech_ind_scaler.fit_transform(technical_indicators)

    assert ohlcv_histories_normalised.shape[0] == next_day_open_values_normalised.shape[0] == technical_indicators_normalised.shape[0]
    return ohlcv_histories_normalised, technical_indicators_normalised, next_day_open_values_normalised, next_day_open_values, y_normaliser


def multiple_csv_to_dataset(test_set_name):
    import os
    ohlcv_histories = 0
    technical_indicators = 0
    next_day_open_values = 0
    for csv_file_path in list(filter(lambda x: x.endswith('daily.csv'), os.listdir('./'))):
        if not csv_file_path == test_set_name:
            print(csv_file_path)
            if type(ohlcv_histories) == int:
                ohlcv_histories, technical_indicators, next_day_open_values, _, _ = csv_to_dataset(csv_file_path)
            else:
                a, b, c, _, _ = csv_to_dataset(csv_file_path)
                ohlcv_histories = np.concatenate((ohlcv_histories, a), 0)
                technical_indicators = np.concatenate((technical_indicators, b), 0)
                next_day_open_values = np.concatenate((next_day_open_values, c), 0)

    ohlcv_train = ohlcv_histories
    tech_ind_train = technical_indicators
    y_train = next_day_open_values

    ohlcv_test, tech_ind_test, y_test, unscaled_y_test, y_normaliser = csv_to_dataset(test_set_name)

    return ohlcv_train, tech_ind_train, y_train, ohlcv_test, tech_ind_test, y_test, unscaled_y_test, y_normaliser